# Cleaning and combining the data

In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

%matplotlib inline

datadir = "../data/"

This notebook will walk through the process of cleaning and combining the data from IPEDS into a data set for analysis. The final form at the end will be a pandas panel (appropriately serialized and stored) which contains the following information:

* for each year between 1984 and 2013 (the items axis)
    * for each institution (the major axis)
        * institutional characteristics:
            1. The IPEDS unit ID
            2. the institution name
            3. the sector (public/private; for profit/nonprofit; less than 2 year/2 year/4+ year)
            4. latitude and longitude (for mapping purposes)
        * Fall Enrollments:
            1. Undergraduate FTE
            2. Ethnicities will go here if I decide I need them
            3. Incoming test scores will go here if I can find them (I've seen them for later years, but not for earlier years)
        * Faculty and staff characteristics:
            1. number of faculty in each academic rank
            2. median salaries
            3. percentages with terminal degrees
            4. size of administrative staff
            5. non-faculty staff salary information (by job type? or something?)
        * Finances
            1. size of the endowment (mostly for private, non-profit institutions)
            2. tuition revenues
            3. State and Federal appropriations/grants
            4. research revenues
            5. instructional costs
            6. other costs
        * Admissions
            1. Number of applicants
            2. Number of admits
            3. number of enrollees
        * Retention
            1. First year retention
            2. other retention numbers, if available
            3. 4 year completion rate
            4. 6 year completion rate
        * Tuition, financial aid and student debt
            1. sticker price (tuition only)
            2. fully loaded cost
            3. discount rate
            4. percentage of students receiving aid
            5. average award amount
            6. average student debt

That's an impressively long list. The variables are spread across several files (and different files for different years). We'll do each category separately.

## Institutional Characteristics

In [2]:
def process(csvfile, columns, charcols):
    raw = pd.read_csv(datadir + csvfile, encoding='latin-1')
    # standardize column names to uppercase without leading or trailing whitespace
    raw.columns = [colname.strip().lower() for colname in raw.columns]
    
    # this is present in all of the files
    prepped = raw.set_index('unitid')
    
    # only transform the columns if we actually have them
    # they may not be present in every year
    intersect = list(set(charcols) & set(prepped.columns))
    for col in intersect:
        prepped[col] = prepped[col].str.strip()
    
    cooked = pd.DataFrame(prepped, columns=columns)
    return cooked

In [3]:
# the variables we want
columns = ['instnm', 'sector', 'act', 'latitude', 'longitud']

# these columns have string values; we want to trim extra whitespace.
charcols = ['instnm', 'act']

# files for institutional characteristics
csvfiles = {1984: 'ic1984.csv',
            1985: 'ic1985.csv',
            1986: 'ic1986_a.csv',
            1987: 'ic1987_a.csv',
            1988: 'ic1988_a.csv',
            1989: 'ic1989_a.csv',
            1990: 'ic90hd.csv',
            1991: 'ic1991_hdr.csv',
            1992: 'ic1992_a.csv',
            1993: 'ic1993_a.csv',
            1994: 'ic1994_a.csv',
            1995: 'ic9596_a.csv',
            1996: 'ic9697_a.csv',
            1997: 'ic9798_hdr.csv',
            1998: 'ic98hdac.csv',
            1999: 'ic99_hd.csv',
            2000: 'fa2000hd.csv',
            2001: 'fa2001hd.csv',
            2002: 'hd2002.csv',
            2003: 'hd2003.csv',
            2004: 'hd2004.csv',
            2005: 'hd2005.csv',
            2006: 'hd2006.csv',
            2007: 'hd2007.csv',
            2008: 'hd2008.csv',
            2009: 'hd2009.csv',
            2010: 'hd2010.csv',
            2011: 'hd2011.csv',
            2012: 'hd2012.csv',
            2013: 'hd2013.csv'}

dataframes = {}
for year in csvfiles.keys():
    dataframes[year] = process(csvfiles[year], columns, charcols)

/Users/cmckay/anaconda/envs/collegestats/lib/python3.4/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (6,214) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)
/Users/cmckay/anaconda/envs/collegestats/lib/python3.4/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)
/Users/cmckay/anaconda/envs/collegestats/lib/python3.4/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (39,40) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)
/Users/cmckay/anaconda/envs/collegestats/lib/python3.4/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (41,42) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)
/Users/cmckay/anaconda/envs/collegestats/lib/python3.4/site-package

### Fix 1986

In 1986, more than 1200 institutions were added to IPEDS, and got the same unitid in the institutional characteristics file.

In [4]:
dataframes[1986].index.get_duplicates()

[247719]

In [5]:
dataframes[1986].ix[247719].describe()

,sector,act,latitude,longitud
count,1264,0,0,0
mean,9,NaN,NaN,NaN
std,0,NaN,NaN,NaN
min,9,NaN,NaN,NaN
25%,9,NaN,NaN,NaN
50%,9,NaN,NaN,NaN
75%,9,NaN,NaN,NaN
max,9,NaN,NaN,NaN


Since they're all sector 9 institutions (which I don't care about too much) the easiest thing is to just drop them from 1986. The ones that show up in later years with correct unitids will continue on from there.

In [6]:
# this would work in pandas 0.17, but unfortunately I'm running 0.16.
#dataframes[1986] = dataframes[1986].reset_index().drop_duplicates(subset='index', keep=False).set_index('index')

dataframes[1986] = dataframes[1986].reset_index().drop_duplicates(subset='unitid').set_index('unitid')

In [7]:
dataframes[1986].index.get_duplicates()

[]

### Deal with NaNs

The columns listing activity, latitude, and longitude are relatively late additions to the data. Activity first appears in 1998, and the location data shows up in 2009. For the years prior to that, I'm going to grab latitudes and longitudes from 2009, and set all of the activities prior to 1998 to 'A', which means 'active'.

In addition to that, the sector designation didn't appear before 1986, and even in 86 and 87 there were a few with undefined sectors. We'll drop those, and copy the sectors from 1986 back to 1984 and 1985.

#### Sector

In [8]:
dataframes[1984]['sector'] = dataframes[1986]['sector']
dataframes[1985]['sector'] = dataframes[1986]['sector']

for year in [1984, 1985, 1986, 1987]:
    dataframes[year] = dataframes[year].dropna(subset=['sector'])

#### Activity

In [9]:
for year in range(1984, 1998):
    dataframes[year]['act'] = 'A'

#### Lat/Lon

In [10]:
for year in range(1984, 2009):
    dataframes[year]['latitude'] = dataframes[2009]['latitude']
    dataframes[year]['longitud'] = dataframes[2009]['longitud']

In [11]:
dataframes[1984].describe()

,sector,latitude,longitud
count,3471.000000,2991.000000,2991.000000
mean,2.691731,38.237865,-89.048261
std,1.513641,5.468945,18.560790
min,0.000000,-14.322592,-170.742769
25%,2.000000,35.006035,-96.071824
50%,2.000000,39.510445,-85.505540
75%,4.000000,41.798333,-77.240769
max,9.000000,64.858048,158.212434


We still have some NaNs, but they're tremendously reduced.

### Subset on Sector

I'm focusing on 4-year, degree granting institutions in this study, so I'm only going to grab the institutions in the approprate sectors (1, 2, and 3). Two-year schools are sectors 4, 5, and 6, and less than 2-year schools are 7, 8, and 9. People interested in looking at those sorts of instiution could easily modify this section to pull out the appropriate data. 

We will do all of the merges below as left joins, so this effectively limits what goes into our final data set.

In [12]:
for year, df in dataframes.items():
    dataframes[year] = df[(df['sector']>0) & (df['sector']<4)]

## Fall enrollments

Unfortunately, what is reported in this section and how it is reported have changed significantly over the 30 year period we're studying. Some distinctions, like age categories, were added, and others, like the year in school, have been removed. Breakdown by ethnicity is available most years, but not all (*e.g.*, 1985).

I'll keep things relatively simple and look at full-time undergraduate men, full-time undergraduate women, part-time undergraduate men, part-time undergraduate women, and grand total of all students. The data is reported as long form panel data, so we'll have to do some grouping to extract the pieces we want into columns for a single unitid.

A little modification to our process routine will allow for this.

In [13]:
def process(csvfile, columns, charcols=[], groupcolumn=None,
            grouplabels=None, collabels=None):
    raw = pd.read_csv(datadir + csvfile, encoding='latin-1')
    # standardize column names to uppercase without leading or trailing whitespace
    raw.columns = [colname.strip().lower() for colname in raw.columns]
    
    # this is present in all of the files
    prepped = raw.set_index('unitid')

    if groupcolumn is not None:
        if grouplabels is None or collabels is None:
            raise KeyError('Specify labels for the desired groups and columns.')
        grouped = prepped.groupby(groupcolumn)
        merged = None
        for value, label in grouplabels:
            groupedcols = {}
            for key, val in collabels.items():
                groupedcols[key] = label + val
            if merged is None:
                merged = pd.DataFrame(grouped.get_group(value), columns=groupedcols.keys())
                merged.columns = groupedcols.values()
            else:
                labeldf = pd.DataFrame(grouped.get_group(value), columns=groupedcols.keys())
                labeldf.columns = groupedcols.values()
                merged = pd.merge(merged, labeldf, left_index=True, right_index=True)
        prepped = merged
        
    
    # only transform the columns if we actually have them
    # they may not be present in every year
    intersect = list(set(charcols) & set(prepped.columns))
    for col in intersect:
        prepped[col] = prepped[col].str.strip()
    
    cooked = pd.DataFrame(prepped, columns=columns)
    return cooked

Now, we need to set up a dictionary to give `process` what it needs: the csv file name, the name of the column to group on, the labels and values of the groups, and a translation for the columns to extract.  I'm choosing to pass these things in rather than hardcode them for a couple of reasons. First, I think this same approach will be necessary when looking at the faculty section, and I'd prefer not to repeat myself. Second, most of these things change over the course of the data.  

I'm actually able to group over the same column in every case, even though in about 2000 another, more detailed breakdown was added. Also, in 2008, the names of the columns I want to extract changed. An additional wrinkle is that up until 2008, they didn't report total students; they kept it broken out in total men and total women. After 2008, they have an additional column which is the sum of the other two. Rather than try to take it if it's there, I'm just going to calculate it after the fact for every case.

In [14]:
# the variables we want to extract:
# FTUGM : Full-time undergraduate men
# FTUGW : Full-time undergraduate women
# PTUGM : Part-time undergraduate men
# PTUGW : Part-time undergraduate women
# ALLM : Total Men (including graduate and professional students)
# ALLW : Total Women (including graduate and professional students)
columns = ['ftugm', 'ftugw', 'ptugm', 'ptugw', 'allm', 'allw']

# No string values in this set
charcols = []

agg84 = {'groupcolumn':'line',
         'grouplabels':((1,'ftug'), ((15,'ptug')), ((29,'all'))),
         'collabels': {'efrace15':'m', 'efrace16':'w'}}

agg86 = {'groupcolumn':'line',
         'grouplabels':((8,'ftug'), ((22,'ptug')), ((29,'all'))),
         'collabels': {'efrace15':'m', 'efrace16':'w'}}

agg08 = {'groupcolumn':'line',
         'grouplabels':((8,'ftug'), ((22,'ptug')), ((29,'all'))),
         'collabels': {'eftotlm':'m', 'eftotlw':'w'}}


# files and columns for fall enrollments
years = {1984: {'csv':'ef1984.csv',
                'aggregation':agg84},
         1985: {'csv':'ef1985.csv',
                'aggregation':agg84},
         1986: {'csv':'ef1986_a.csv',
                'aggregation':agg86},
         1987: {'csv':'ef1987_a.csv',
                'aggregation':agg86},
         1988: {'csv':'ef1988_a.csv',
                'aggregation':agg86},
         1989: {'csv':'ef1989_a.csv',
                'aggregation':agg86},
         1990: {'csv':'ef90_a.csv',
                'aggregation':agg86},
         1991: {'csv':'ef1991_a.csv',
                'aggregation':agg86},
         1992: {'csv':'ef1992_a.csv',
                'aggregation':agg86},
         1993: {'csv':'ef1993_a.csv',
                'aggregation':agg86},
         1994: {'csv':'ef1994_anr.csv',
                'aggregation':agg86},
         1995: {'csv':'ef95_anr.csv',
                'aggregation':agg86},
         1996: {'csv':'ef96_anr.csv',
                'aggregation':agg86},
         1997: {'csv':'ef97_anr.csv',
                'aggregation':agg86},
         1998: {'csv':'ef98_anr.csv',
                'aggregation':agg86},
         1999: {'csv':'ef99_anr.csv',
                'aggregation':agg86},
         2000: {'csv':'ef2000a.csv',
                'aggregation':agg86},
         2001: {'csv':'ef2001a.csv',
                'aggregation':agg86},
         2002: {'csv':'ef2002a.csv',
                'aggregation':agg86},
         2003: {'csv':'ef2003a.csv',
                'aggregation':agg86},
         2004: {'csv':'ef2004a.csv',
                'aggregation':agg86},
         2005: {'csv':'ef2005a.csv',
                'aggregation':agg86},
         2006: {'csv':'ef2006a.csv',
                'aggregation':agg86},
         2007: {'csv':'ef2007a.csv',
                'aggregation':agg86},
         2008: {'csv':'ef2008a.csv',
                'aggregation':agg08},
         2009: {'csv':'ef2009a.csv',
                'aggregation':agg08},
         2010: {'csv':'ef2010a.csv',
                'aggregation':agg08},
         2011: {'csv':'ef2011a.csv',
                'aggregation':agg08},
         2012: {'csv':'ef2012a.csv',
                'aggregation':agg08},
         2013: {'csv':'ef2013a.csv',
                'aggregation':agg08},
}

In [15]:
efdataframes = {}
for year, info in years.items():
    efdataframes[year] = process(info['csv'], columns, charcols,
                                groupcolumn=info['aggregation']['groupcolumn'],
                                grouplabels=info['aggregation']['grouplabels'],
                                collabels=info['aggregation']['collabels'])

### Compute totals and ratios

In [16]:
for enroldf in efdataframes.values():
    # totals for full time, part time, and all students
    enroldf['ftugt'] = enroldf['ftugm'] + enroldf['ftugw']
    enroldf['ptugt'] = enroldf['ptugm'] + enroldf['ptugw']
    enroldf['allt'] = enroldf['allm'] + enroldf['allw']
    
    # male/total fraction for FTUG:
    enroldf['ftmf'] = enroldf['ftugm']/enroldf['ftugt']
    
    # FT undergraduate fraction:
    enroldf['ugfrac'] = enroldf['ftugt']/enroldf['allt']

### Merge into existing dataframes

In [17]:
for year, yeardf in dataframes.items():
    dataframes[year] = pd.merge(yeardf, efdataframes[year], how='left', left_index=True, right_index=True)

In [18]:
dataframes[2013].head()

,instnm,sector,act,latitude,longitud,ftugm,ftugw,ptugm,ptugw,allm,allw,ftugt,ptugt,allt,ftmf,ugfrac
unitid,,,,,,,,,,,,,,,,
100654,Alabama A & M University,1,A,34.783368,-86.568502,1836,1963,133,119,2268,2752,3799,252,5020,0.483285,0.756773
100663,University of Alabama at Birmingham,1,A,33.502230,-86.809170,3501,4856,1279,1866,7309,11259,8357,3145,18568,0.418930,0.450075
100690,Amridge University,2,A,32.362609,-86.174010,74,128,52,68,264,367,202,120,631,0.366337,0.320127
100706,University of Alabama in Huntsville,1,A,34.722818,-86.638420,2331,1906,846,613,4136,3240,4237,1459,7376,0.550153,0.574431
100724,Alabama State University,1,A,32.364317,-86.295677,1975,2897,213,271,2399,3676,4872,484,6075,0.405378,0.801975


## Faculty and staff

IPEDS has three different surveys that deal with faculty and staff:
* Instructional staff/Salaries.  This survey is the most complete (in terms of years of availability), but has some significant drawbacks. The 2012 and 2013 surveys have non-instructional staff included, but none of the other years do. The surveys between 2005 and 2011 include people on contracts less than 9 months (*i.e.*, adjuncts) but other years don't. Fringe benefits were included up until 2010, but apparently not after that. Furthermore, 9 and 12 month contracts are reported separately. While there may be some deeply meaningful reason for doing so, it makes things a little more difficult for me.
* Fall Staff. Available from 1987 onward in odd years, and every year after 2001. This includes noninstructional staff in a whole host of categories, but doesn't include any salary data.
* Employees by Assigned Position. From 2001 on. Has many more categories than either of the others. Includes tenure status but not faculty rank.

If I want to cover the whole range of years (which I do), I can get closest by using the first of these, which means largely ignoring noninstructional staff and adjuncts and dealing with the 9/12 month issues.  I'll do that for a first pass, at least.

The files seem to come in two flavors: wide and long. The long flavor has a variable for academic rank (and another for contract type) which we can group over.  The wide flavor just has all of the variables in a single row.
* wide: 1987, 1989-1998 
* long: 1984, 1985, 1999, 2001-2013 
* missing: 1986, 1988, 2000

We process the long flavor files like the fall enrolment, above.  The wide ones are easier; they're like the institutional characteristics we did first.

The last thing to do before we actually process is to decide which columns we want to keep. We have two sexes, between two and four contract durations, six academic ranks (full, associate, assistant, instructor, lecturer, and no rank), and three tenure statuses (tenured, tenure track, and non-tenure track), for a total of up to 144 columns. That's clearly too many, and in fact, some of the years don't have that many.

If I ignore tenure status (which isn't reported every year, anyway), and gender distribution (there are people better qualified than I to study diversity issues), and contract length, then I'm down to six categories with two variables each (number of people and salary outlay).  Monthly salary may be a more meaningful number, but I don't really care at this point.

#### wide format years

In [19]:
# wide format
# 1987-1997
aggregation = {'fullcount':['a4', 'a39', 'a79', 'a114'],
               'fulloutlay':['a5', 'a40', 'a80', 'a115'],
               'assoccount':['a9', 'a44', 'a84', 'a119'],
               'assocoutlay':['a10', 'a45', 'a85', 'a120'],
               'assistcount':['a14', 'a49', 'a89', 'a124'],
               'assistoutlay':['a15', 'a50', 'a90', 'a125'],
               'instrcount':['a19', 'a54', 'a94', 'a129'],
               'instroutlay':['a20', 'a55', 'a95', 'a130'],
               'lectcount':['a24', 'a59', 'a99', 'a134'],
               'lectoutlay':['a25', 'a60', 'a100', 'a135'],
               'norankcount':['a29', 'a64', 'a104', 'a139'],
               'norankoutlay':['a30', 'a65', 'a105', 'a140']}

# 1998
agg98 = {'fullcount':['saa014', 'saa084', 'saa164', 'saa234'],
         'fulloutlay':['saa015', 'saa085', 'saa165', 'saa235'],
         'assoccount':['saa024', 'saa094', 'saa174', 'saa244'],
         'assocoutlay':['saa025', 'saa095', 'saa175', 'saa245'],
         'assistcount':['saa034', 'saa104', 'saa184', 'saa254'],
         'assistoutlay':['saa035', 'saa105', 'saa185', 'saa255'],
         'instrcount':['saa044', 'saa114', 'saa194', 'saa264'],
         'instroutlay':['saa045', 'saa115', 'saa195', 'saa265'],
         'lectcount':['saa054', 'saa124', 'saa204', 'saa274'],
         'lectoutlay':['saa055', 'saa125', 'saa205', 'saa275'],
         'norankcount':['saa064', 'saa134', 'saa214', 'saa284'],
         'norankoutlay':['saa065', 'saa135', 'saa215', 'saa285']}

wideyears = {1987: {'csv':'sal1987_a.csv',
                   'agg':aggregation},
             1989: {'csv':'sal1989_a.csv',
                   'agg':aggregation},
             1990: {'csv':'sal90_a.csv',
                   'agg':aggregation},
             1991: {'csv':'sal1991_a.csv',
                   'agg':aggregation},
             1992: {'csv':'sal1992_a.csv',
                   'agg':aggregation},
             1993: {'csv':'sal1993_a.csv',
                   'agg':aggregation},
             1994: {'csv':'sal1994_a.csv',
                   'agg':aggregation},
             1995: {'csv':'sal95_a_1.csv',
                   'agg':aggregation},
             1996: {'csv':'sal96_a_1.csv',
                   'agg':aggregation},
             1997: {'csv':'sal97_a.csv',
                   'agg':aggregation},
             1998: {'csv':'sal98_a.csv',
                   'agg':agg98}}

def process_wide(csvfile, aggregation):
    raw = pd.read_csv(datadir + csvfile, encoding='latin-1')
    # standardize column names to lowercase without leading or trailing whitespace
    raw.columns = [colname.strip().lower() for colname in raw.columns]
    
    # this is present in all of the files
    prepped = raw.set_index('unitid')
    
    for aggregated, aggcols in aggregation.items():
        prepped[aggregated] = prepped[aggcols].sum(axis=1)
    
    cooked = pd.DataFrame(prepped, columns=aggregation.keys()).fillna(0)
    return cooked

In [20]:
facultydfs = {}
for year, yeardict in wideyears.items():
    facultydfs[year] = process_wide(yeardict['csv'], yeardict['agg'])

#### Create empty data frames for missing years

In [21]:
missingyears = [1986, 1988, 2000]
for year in missingyears:
    facultydfs[year] = pd.DataFrame(columns=aggregation.keys())

#### do the long format years

In [22]:
agg84 = {'count':['saa1', 'saa4'],
         'outlay':['saa2', 'saa5'],
         'group':'line',
         'vals':{'full':[1,8],
                 'assoc':[2,9],
                 'assist':[3,10],
                 'instr':[4,11],
                 'lect':[5,12],
                 'norank':[6,13]}}

# in 1999, the survey separated academic rank and contract length into 
# two line items
agg99 = {'count':['empcntm', 'empcntw'],
         'outlay':['outlaym', 'outlayw'],
         'group':['contract','arank'],
         'vals':{'full':[(1,1), (2,1)],
                 'assoc':[(1,2), (2,2)],
                 'assist':[(1,3), (2,3)],
                 'instr':[(1,4), (2,4)],
                 'lect':[(1,5), (2,5)],
                 'norank':[(1,6), (2,6)]}}

# the only survey available from 2001 is a summary, which doesn't separate women and men.
# otherwise, it's the same as 99.
agg01 = {'count':['empcount',],
         'outlay':['outlays',],
         'group':['contract','arank'],
         'vals':{'full':[(1,1), (2,1)],
                 'assoc':[(1,2), (2,2)],
                 'assist':[(1,3), (2,3)],
                 'instr':[(1,4), (2,4)],
                 'lect':[(1,5), (2,5)],
                 'norank':[(1,6), (2,6)]}}

# in 2012, the contract length went wide while the academic rank stayed long.
# also, contract length was split into four categories instead of two.
# fortunately, they also include an aggregated column.
agg12 = {'count':['satotlt',],
         'outlay':['saoutlt',],
         'group':'arank',
         'vals':{'full':[1,],
                 'assoc':[2,],
                 'assist':[3,],
                 'instr':[4,],
                 'lect':[5,],
                 'norank':[6,]}}

# long format
longyears = {1984:{'csv':'sal1984_a.csv',
                   'agg':agg84},
             1985:{'csv':'sal1985_a.csv',
                   'agg':agg84},
             1999:{'csv':'sal1999_a.csv',
                   'agg':agg99},
             2001:{'csv':'sal2001_a_s.csv',
                   'agg':agg01},
             2002:{'csv':'sal2002_a.csv',
                   'agg':agg99},
             2003:{'csv':'sal2003_a.csv',
                   'agg':agg99},
             2004:{'csv':'sal2004_a.csv',
                   'agg':agg99},
             2005:{'csv':'sal2005_a.csv',
                   'agg':agg99},
             2006:{'csv':'sal2006_a.csv',
                   'agg':agg99},
             2007:{'csv':'sal2007_a.csv',
                   'agg':agg99},
             2008:{'csv':'sal2008_a.csv',
                   'agg':agg99},
             2009:{'csv':'sal2009_a.csv',
                   'agg':agg99},
             2010:{'csv':'sal2010_a.csv',
                   'agg':agg99},
             2011:{'csv':'sal2011_a.csv',
                   'agg':agg99},
             2012:{'csv':'sal2012_is.csv',
                   'agg':agg12},
             2013:{'csv':'sal2013_is.csv',
                   'agg':agg12}}


def process_long(csvfile, aggregation):
    raw = pd.read_csv(datadir + csvfile, encoding='latin-1')
    # standardize column names to lowercase without leading or trailing whitespace
    raw.columns = [colname.strip().lower() for colname in raw.columns]
    
    # this is present in all of the files
    prepped = raw.set_index('unitid')
    
    grouped = prepped.groupby(aggregation['group'])
    collected = {}
    for rank,val in aggregation['vals'].items():
        groups = []
        for subgroup in val:
            groups.append(grouped.get_group(subgroup))
            
        catted = pd.concat(groups)
        unitgroups = catted.groupby(level=0)
        collected[rank+"count"] = unitgroups.sum()[aggregation['count']].sum(axis=1)
        collected[rank+"outlay"] = unitgroups.sum()[aggregation['outlay']].sum(axis=1)
        
    cooked = pd.DataFrame(collected).fillna(0)
    return cooked

In [23]:
for year, yeardict in longyears.items():
    facultydfs[year] = process_long(yeardict['csv'], yeardict['agg'])

### Totals and ratios

I want the total number of faculty, and the student/faculty ratio.

In [25]:
allcounts = ['fullcount', 'assoccount', 'assistcount', 'instrcount', 'lectcount', 'norankcount']

for year, facdf in facultydfs.items():
    facdf['totalfac'] = facdf[allcounts].sum(axis=1)
    facdf['studfacratio'] = dataframes[year]['allt']/facdf['totalfac']

### Merge into existing dataframes

In [26]:
for year, yeardf in dataframes.items():
    dataframes[year] = pd.merge(yeardf, facultydfs[year], how='left', left_index=True, right_index=True)

## Finances

This is a complicated topic, and I am by no means an expert.  As I see it, there are four important categories of numbers in this section:
1. income
2. expenses
3. assets
4. liabilities

Unfortunately, the three major types of institution (public, non-profit private, and for-profit private) have significantly different sources of income, and often very different collections of assets and liabilities, as well.  That makes comparisons difficult.  From about 1997 on the three categories of institution have their data in separate files. Before that, the organization of the data is kind of a mess. The first couple years of our range drop everything into a single file, but through most of the 90's, there are a wide array of different files for each year.

I'm going to have to select a subset of the possible variables so that I don't drown in them. (There are about 200 different variables for nonprofit institutions in the 2013 data; public institutions (160) and for-profit institutions (40) have fewer). For now, I will focus on six numbers: totals for income, expenses, and debt; amount of income from tuition and fees, value of the endowment at the end of the year, and the amount expended on instructional costs.

Assets are kind of a sticky area. Things like land and physical plant are very illiquid, and depend strongly on local market values. On the other hand, for-profit institutions don't have an endowment, as such, but I care less about their finances at this point. If I decide I need some other measure of total assets, or something additional for the for-profit institutions, I'll go back and refactor.

In [27]:
mapping84 = {'totalrevenue':'a20', 
             'tuitionrevenue':'a01',
             'instructionexpense':'b01',
             'totalexpense':'b19',
             'debt':'d04',
             'endowment':'f64'}

# total debt and endowment don't seem to be reported in 88.
mapping88 = {'totalrevenue':'a163', 
             'tuitionrevenue':'a013',
             'instructionexpense':'b013',
             'totalexpense':'b223'}

single_files = {1984:{'csv':'f1984.csv',
                     'mapping':mapping84},
               1985:{'csv':'f1985.csv',
                     'mapping':mapping84},
               1986:{'csv':'f1986.csv',
                     'mapping':mapping84},
               1988:{'csv':'f1988.csv',
                     'mapping':mapping88}}

def process_single_financial(csvfile, mapping):
    # index col is needed for 1997, for reasons I don't fully understand
    raw = pd.read_csv(datadir + csvfile, encoding='latin-1', index_col=False)
    # standardize column names to uppercase without leading or trailing whitespace
    raw.columns = [colname.strip().lower() for colname in raw.columns]
    
    # this is present in all of the files
    prepped = raw.set_index('unitid')
    for key, colname in mapping.items():
        prepped[key] = prepped[colname]
    cooked = pd.DataFrame(prepped, columns=mapping.keys())
    return cooked

In [28]:
financedfs = {}
for fyear, mapping in single_files.items():
    financedfs[fyear] = process_single_financial(mapping['csv'], mapping['mapping'])

In [29]:
expenses87 = {'totalexpense':'b223',
              'instructionexpense':'b013'}
income87 = {'totalrevenue':'a163',
            'tuitionrevenue':'a013'}
debt89 = {'debt':'g041'}
endowment89 = {'endowment':'h021'}

debt93 = {'debt':'g04'}

multi_by_category = {1987:{'f1987_a.csv':income87,
                           'f1987_b.csv':expenses87},
                     1989:{'f1989_a.csv':income87,
                           'f1989_b.csv':expenses87,
                           'f1989_g.csv':debt89,
                           'f1989_h.csv':endowment89},
                     1990:{'f8990_a.csv':income87,
                           'f8990_b.csv':expenses87},
                     1991:{'f1991_a.csv':income87,
                           'f1991_b.csv':expenses87,
                           'f1991_g.csv':debt89,
                           'f1991_h.csv':endowment89},
                     1992:{'f1992_a.csv':income87,
                           'f1992_b.csv':expenses87,
                           'f1992_g.csv':debt89,
                           'f1992_h.csv':endowment89},
                     1993:{'f1993_a.csv':income87,
                           'f1993_b.csv':expenses87,
                           'f1993_g.csv':debt93,
                           'f1993_h.csv':endowment89},
                     1994:{'f1994_a.csv':income87,
                           'f1994_b.csv':expenses87,
                           'f1994_g.csv':debt93,
                           'f1994_h.csv':endowment89},
                     1995:{'f9495_a.csv':income87,
                           'f9495_b.csv':expenses87,
                           'f9495_g.csv':debt93,
                           'f9495_h.csv':endowment89},
                     1996:{'f9596_a.csv':income87,
                           'f9596_b.csv':expenses87,
                           'f9596_g.csv':debt93,
                           'f9596_h.csv':endowment89}}

def process_multi_cat_financial(csvdict):
    merged = None
    for csvfile, mapping in csvdict.items():
        if merged is None:
            merged = process_single_financial(csvfile, mapping)
        else:
            temp = process_single_financial(csvfile, mapping)
            merged = pd.merge(merged, temp, left_index=True, right_index=True)
    return merged

In [30]:
for year, cvsdict in multi_by_category.items():
    financedfs[year] = process_multi_cat_financial(cvsdict)

In [31]:
public97 = {'totalexpense':'b223',
            'instructionexpense':'b013',
            'totalrevenue':'a163',
            'tuitionrevenue':'a013',
            'debt':'g04',
            'endowment':'h021'}

private97 = {'totalexpense':'fb12_1',
             'instructionexpense':'fb01_1',
             'totalrevenue':'fa17_1',
             'tuitionrevenue':'fa01_1'}

nonprofit98 = {'totalexpense':'f2b12_1',
               'instructionexpense':'f2b01_1',
               'totalrevenue':'f2a17_1',
               'tuitionrevenue':'f2a01_1',
               'endowment':'f2d05'}

forprofit98 = {'totalexpense':'f3b10_1',
               'instructionexpense':'f3b01_1',
               'totalrevenue':'f3a10',
               'tuitionrevenue':'f3a01'}

nonprofit00 = {'totalrevenue':'f2b01',
               'tuitionrevenue':'f2d01',
               'totalexpense':'f2b02',
               'instructionexpense':'f2e011',
               'endowment':'f2a01'}

forprofit00 = {'totalexpense':'f3b02',
               'instructionexpense':'f3e01',
               'totalrevenue':'f3b01',
               'tuitionrevenue':'f3d01'}

public02 = {'totalexpense':'f1c151',
            'instructionexpense':'f1c011',
            'totalrevenue':'f1b09',
            'tuitionrevenue':'f1b01'}

public10 = {'totalexpense':'f1c191',
            'instructionexpense':'f1c011',
            'totalrevenue':'f1b09',
            'tuitionrevenue':'f1b01'}

multi_by_sector = {1997:{'f9697_f1.csv':public97,
                         'f9697_f2.csv':private97},
                   1998:{'f9798_f1.csv':public97,
                         'f9798_f2.csv':nonprofit98,
                         'f9798_f3.csv':forprofit98},
                   1999:{'f9899_f1.csv':public97,
                         'f9899_f2.csv':nonprofit98,
                         'f9899_f3.csv':forprofit98},
                   2000:{'f9900_f1.csv':public97,
                         'f9900f2.csv':nonprofit00,
                         'f9900f3.csv':forprofit00},
                   2001:{'f0001_f1.csv':public97,
                         'f0001_f2.csv':nonprofit00,
                         'f0001_f3.csv':forprofit00},
                   2002:{'f0102_f1.csv':public97,
                         'f0102_f1a.csv':public02,
                         'f0102_f2.csv':nonprofit00,
                         'f0102_f3.csv':forprofit00},
                   2003:{'f0203_f1.csv':public97,
                         'f0203_f1a.csv':public02,
                         'f0203_f2.csv':nonprofit00,
                         'f0203_f3.csv':forprofit00},
                   2004:{'f0304_f1a.csv':public02,
                         'f0304_f2.csv':nonprofit00,
                         'f0304_f3.csv':forprofit00},
                   2005:{'f0405_f1a.csv':public02,
                         'f0405_f2.csv':nonprofit00,
                         'f0405_f3.csv':forprofit00},
                   2006:{'f0506_f1a.csv':public02,
                         'f0506_f2.csv':nonprofit00,
                         'f0506_f3.csv':forprofit00},
                   2007:{'f0607_f1a.csv':public02,
                         'f0607_f2.csv':nonprofit00,
                         'f0607_f3.csv':forprofit00},
                   2008:{'f0708_f1a.csv':public02,
                         'f0708_f2.csv':nonprofit00,
                         'f0708_f3.csv':forprofit00},
                   2009:{'f0809_f1a.csv':public02,
                         'f0809_f2.csv':nonprofit00,
                         'f0809_f3.csv':forprofit00},
                   2010:{'f0910_f1a.csv':public10,
                         'f0910_f2.csv':nonprofit00,
                         'f0910_f3.csv':forprofit00},
                   2011:{'f1011_f1a.csv':public10,
                         'f1011_f2.csv':nonprofit00,
                         'f1011_f3.csv':forprofit00},
                   2012:{'f1112_f1a.csv':public10,
                         'f1112_f2.csv':nonprofit00,
                         'f1112_f3.csv':forprofit00},
                   2013:{'f1213_f1a.csv':public10,
                         'f1213_f2.csv':nonprofit00,
                         'f1213_f3.csv':forprofit00}}


def process_multi_sector_financial(csvdict):
    dfs_to_merge = []
    for csvfile, mapping in csvdict.items():
        dfs_to_merge.append(process_single_financial(csvfile, mapping))
    merged = pd.concat(dfs_to_merge)
    return merged

In [32]:
for year, cvsdict in multi_by_sector.items():
    financedfs[year] = process_multi_sector_financial(cvsdict)

### Calculate some ratios

There are two ratios in particular I'm interested in: the fraction of total revenue made up by tuition revenues (tuition dependence) and the fraction of total expenses made up by instruction (teaching centeredness).

In [36]:
for year, df in financedfs.items():
    try:
        df['tuitiondependence'] = df['tuitionrevenue']/df['totalrevenue']
    except ZeroDivisionError:
        df['tuitiondependence'] = -1.0
    try:
        df['teachingcenteredness'] = df['instructionexpense']/df['totalexpense']
    except ZeroDivisionError:
        df['teachingcenteredness'] = -1.0

### Merge

In [33]:
for year, yeardf in dataframes.items():
    dataframes[year] = pd.merge(yeardf, financedfs[year], how='left', left_index=True, right_index=True)

## Admissions

## Retention

## Tuition, financial aid, and student debt

# Serialize and Store

We have a lot of options for serializing this data set. If we just want to store the data, without adding any functionality, we can use pickle, json, or even csv. Of these three, json might be the most versatile, simply because it allows more structure than csv but is more cross-language than pickle. An additional advantage to json is that if we're going to use something like D3 for visualization, the data is already packaged.

Dataframes lend themselves to a SQL serialization, but we really want the third dimension in this dataset, and that doesn't seem like a natural fit for SQL. So, I'm going to go with HDF. It's fast, it handles large datasets well, and it can do some selection on the data, meaning that we don't have to retrieve everything if we're just looking at a few variables (which wouldn't be possible with json).

We first have to combine the collection of dataframes into a panel, and then serialize it.

In [34]:
collegestats = pd.Panel(dataframes)

collegestats.to_hdf('../data/collegestats.h5', 'collegestats', mode='w')

/Users/cmckay/anaconda/envs/collegestats/lib/python3.4/site-packages/pandas/io/pytables.py:2577: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block0_values] [items->[1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013]]

  warnings.warn(ws, PerformanceWarning)
